<a href="https://colab.research.google.com/github/vitor-marinho-fjp/indicadores_MUNIC/blob/main/indicadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

# IMRS Esporte ------------------------------------------------------------

# Ano para o qual deseja-se obter os dados (ou seja, coloca-se o ano anterior ao atual)
ano_dados = 2021

# Bases -------------------------------------------------------------------

# Aqui é realizada a leitura dos arquivos em formato .xlsx. Os arquivos deverão estar na mesma pasta em que o
# script está salvo.
dados_imrs = pd.read_excel("/content/drive/MyDrive/imrs_esporte/IMRS2021 - BASE ESPORTE.xlsx", sheet_name=0)
dados_icms = pd.read_excel("/content/drive/MyDrive/imrs_esporte/Dados_Basicos_Esportes_2022.xlsx", sheet_name=0).clean_names()
dados_quadra = pd.read_excel("/content/drive/MyDrive/imrs_esporte/ProcQuadras_2021.xlsx", sheet_name=0)
dados_munic = pd.read_excel("/content/drive/MyDrive/imrs_esporte/Base_MUNIC_2021.xlsx", sheet_name=6).rename(columns={1: "IBGE7"}).query("UF=='MG'")

# Dados MUNIC -------------------------------------------------------------

dados_munic = pd.read_excel("/content/drive/MyDrive/imrs_esporte/Base_MUNIC_2021.xlsx", sheet_name=7).rename(columns={1: "IBGE7"}).query("UF=='MG'").assign(ANO=2021)

indicadores_munic = dados_munic[["IBGE7", "ANO", "Mesp22", "Mesp221", "Mesp18", "Mesp10", "Mesp01"]].rename(
    columns={"Mesp22": "L_EQUI", "Mesp221": "L_QUANT", "Mesp18": "L_PARTESP", "Mesp10": "L_CONSESP", "Mesp01": "L_ORGESP"})

l_convesp = dados_munic.assign(L_CONVESP=lambda x: np.where(
    (x["Mesp161"] == "Sim") | (x["Mesp162"] == "Sim") | (x["Mesp163"] == "Sim") | (x["Mesp164"] == "Sim"), "Sim", "Não"
)).loc[:, ["IBGE7", "L_CONVESP"]]

indicadores_munic = pd.merge(indicadores_munic, l_convesp, on="IBGE7").drop(columns=["L_CONVESP"])

# criando CHAVE
indicadores_munic["IBGE6"] = indicadores_munic["IBGE7"].astype(str).str[:-1].astype(float)
indicadores_munic["CHAVE"] = indicadores_munic["ANO"].astype(str) + indicadores_munic["IBGE6"].astype(str)

# ICMS --------------------------------------------------------------------

# Reconstrói o código IBGE
indicadores_icms = dados_icms.assign(IBGE6=lambda x: x["ibge"].astype(str).str.zfill(6).astype(float))

# Cria a coluna de chave, usando o ano anterior
indicadores_icms["CHAVE"] = (ano_dados - 1) * 1000000 + indicadores_icms["IBGE6"]

# Assinalando não para os valores ausentes
indicadores_icms["conselho_de_esportes"] = np.where(indicadores_icms["conselho_de_esportes"].isna(), "NÃO",
                                                   indicadores_icms["conselho_de_esportes"])

# Cálculo do indicador L_PROGE
indicadores_icms["L_PROGE"] = np.where(
    (indicadores_icms["conselho_de_esportes"] == "SIM") & (~indicadores_icms["pontuacao_municipio_soma_das_atividades_do_municipio"].isna()),
    indicadores_icms["pontuacao_municipio_soma_das_atividades_do_municipio"] / indicadores_icms["peso_da_rcl"], 0)

# L_ILRHE - Participação percentual em atividades esportivas habilitadas no critério esporte do ICMS Solidário
indicadores_icms["L_ILRHE"] = np.where(
    (indicadores_icms["conselho_de_esportes"] == "SIM") & (~indicadores_icms["pontuacao_municipio_soma_das_atividades_do_municipio"].isna()),
    (indicadores_icms["pontuacao_municipio_soma_das_atividades_do_municipio"] / indicadores_icms["soma_pontuacao_municipios_mg"]) * 100, 0)

# Selecionar colunas relevantes
indicadores_icms = indicadores_icms[["IBGE6", "L_PROGE", "L_ILRHE"]]

# Juntar bases
indicadores = pd.merge(indicadores_munic, indicadores_icms, on="IBGE6")

# IDEB --------------------------------------------------------------------

# L_ESPESC - Percentual de alunos em escolas com quadra de esporte
dados_quadra.columns = ["IBGE7", "pquadras"]
ind_qdr = pd.merge(indicadores, dados_quadra, on="IBGE7")
indicadores = ind_qdr.assign(L_ESPESC=ind_qdr["pquadras"]).drop(columns=["pquadras"])

# Ajustar e exporta a base ------------------------------------------------

indicadores = indicadores[["CHAVE", "IBGE6", "IBGE7", "ANO", "L_EQUI", "L_QUANT", "L_PARTESP", "L_PROGE", "L_ESPESC", "L_ILRHE",
                           "L_CONSESP", "L_ORGESP", "L_CONVESP"]]

# Arredondando todas as colunas numéricas para duas casas decimais
indicadores = indicadores.round(decimals=2)

# Juntar com a base histórica do IMRS
dados_imrs = pd.concat([dados_imrs, indicadores])

# Exportar apenas os indicadores calculados
indicadores.to_excel("/content/drive/MyDrive/imrs_esporte/imrs_esporte_2021.xlsx", index=False)


AttributeError: 'DataFrame' object has no attribute 'clean_names'